In [16]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [17]:
import torch
print("GPU kullanılıyor mu?", torch.cuda.is_available())
print("GPU ismi:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Yok")


GPU kullanılıyor mu? True
GPU ismi: Tesla P100-PCIE-16GB


In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [19]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [20]:
!huggingface-cli login --token $secret_hf

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `colab-mistral` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab-mistral`


In [21]:
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [22]:
from datasets import load_dataset

base_model = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
dataset_name = "/kaggle/input/turkish-joke-dataset"
new_model = "mistral_7b_turkish_joke"

In [23]:
from datasets import load_dataset

# Dataset'i yükle
dataset = load_dataset(dataset_name, split="train")

# Her örneği instruction formatına çevir
def format_example(example):
    return {
        "text": f"<s>[INST] Fıkra yazar mısın? [/INST]\n{example['text']}</s>"
    }

formatted_dataset = dataset.map(format_example)

In [24]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [26]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [27]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=25,
    learning_rate=1e-4,  # veya 5e-5
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="wandb"
)

In [28]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_dataset = dataset.map(tokenize)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

In [29]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_arguments
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.245600
50,1.068900
75,0.962300
100,0.916200
125,0.856200
150,0.851400
175,0.811100
200,0.772300
225,0.844400
250,0.782900


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

In [ ]:
# Model'i kaydetme

trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
!zip -r mistral_7b_turkish_joke.zip mistral_7b_turkish_joke

In [ ]:
# Model'i yayinlama
'''
trainer.model.push_to_hub(new_model, use_temp_dir=False)
'''

In [ ]:
import logging
from transformers import pipeline
from transformers import pipeline, logging

# Gürültüyü azalt
logging.set_verbosity(logging.CRITICAL)

prompt = "Fıkra yazar mısın?"

# Pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,
    temperature=0.3,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.5
)

# Formatlı prompt ile çıktı al
result = pipe(f"<s>[INST] {prompt} [/INST]\n")

# Sadece cevabı alalım, başlıkları filtreleyelim
generated_text = result[0]['generated_text']

# [INST] içeriğini ve </s> sonrası fazlalıkları kes
if "[/INST]" in generated_text:
    generated_text = generated_text.split("[/INST]")[-1].strip()
if "</s>" in generated_text:
    generated_text = generated_text.split("</s>")[0].strip()

print(generated_text)